In [1]:
!nvidia-smi

Sun Mar  9 02:03:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:81:00.0 Off |                    0 |
| N/A   32C    P0             25W /  250W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import flair
import matplotlib.pyplot as plt
import os
import pandas as pd

from codealltag_data_processor_v2025 import CodealltagDataProcessor
from flair.data import Corpus, Dictionary
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.nn import Model
from flair.trainers import ModelTrainer
from pathlib import Path

In [3]:
flair.cache_root = Path(os.path.join(*['/home', 's81481', '.flair']))

In [4]:
cdp_2022 = CodealltagDataProcessor(data_version='20220513', config_path=['codealltag_data_processor.yml'])

In [5]:
model_tag = "G29B" # "MT5" | "L318B" | "G29B"

In [6]:
def prepare_flair_corpus(data_folder: Path) -> CSVClassificationCorpus:
    
    train_df[['TextTokenized', 'Type']].to_csv(f"{data_folder}/train.csv", 
                                               sep='\t', 
                                               index=False, 
                                               header=['text', 'label'])
    
    dev_df[['TextTokenized', 'Type']].to_csv(f"{data_folder}/dev.csv", 
                                             sep='\t', 
                                             index=False, 
                                             header=['text', 'label'])
    
    test_df[['TextTokenized', 'Type']].to_csv(f"{data_folder}/test.csv", 
                                              sep='\t', 
                                              index=False, 
                                              header=['text', 'label'])
    
    column_name_map = {0: "text", 1: "label"}  
    label_type = "label"
    corpus = CSVClassificationCorpus(
        data_folder,
        column_name_map, 
        skip_header=True, 
        delimiter="\t",
        label_type=label_type,
        train_file='train.csv',
        dev_file='dev.csv',
        test_file='test.csv'
    )
    return corpus

In [7]:
for i in range(0, 5):
    k = i + 1
    
    syntheticity_data = pd.read_csv(f"data_syntheticity_dataset_3000_{model_tag}.csv", index_col=0)
    dataset = cdp_2022.get_train_dev_test_datasetdict_for_syntheticity_dataset(syntheticity_data, 500, k)

    train_df = dataset["train"].to_pandas()
    dev_df = dataset["dev"].to_pandas()
    test_df = dataset["test"].to_pandas()
    
    tmp_data_dir = Path("tmp_data_dir")
    tmp_data_dir.mkdir(parents=True, exist_ok=True)
    for file in tmp_data_dir.glob("*"):
        if file.is_file():
            file.unlink()
    
    model_dir_path = os.path.join(*["logs", "PDSC", "GELECTRA", model_tag, str(len(syntheticity_data)//1000)+"K", "k"+str(k)])
    os.makedirs(model_dir_path, exist_ok=True)
    
    corpus: Corpus = prepare_flair_corpus(data_folder=tmp_data_dir)
    label_dict: Dictionary = corpus.make_label_dictionary(label_type="label")
    classifier = TextClassifier(
        document_embeddings=TransformerDocumentEmbeddings('deepset/gelectra-large', fine_tune=True),
        label_dictionary=label_dict,
        label_type='label',
        multi_label=False
    )
    trainer: ModelTrainer = ModelTrainer(classifier, corpus)
    trainer.fine_tune(
        model_dir_path,
        learning_rate=5.0e-6,
        mini_batch_size=4,
        max_epochs=10,
        checkpoint=True,
        write_weights=True,
        use_tensorboard=True,
        tensorboard_log_dir=model_dir_path,
        save_final_model=False,
        use_final_model_for_eval=False
    )

2025-03-09 02:06:16,599 Reading data from tmp_data_dir
2025-03-09 02:06:16,599 Train: tmp_data_dir/train.csv
2025-03-09 02:06:16,599 Dev: tmp_data_dir/dev.csv
2025-03-09 02:06:16,600 Test: tmp_data_dir/test.csv
2025-03-09 02:06:16,613 Computing label dictionary. Progress:


2000it [00:01, 1654.57it/s]

2025-03-09 02:06:17,848 Dictionary created for label 'label' with 3 values: ORIG (seen 1000 times), PSEUD (seen 1000 times)



/home/s81481/pseugc/lib/python3.9/site-packages/transformers/modeling_utils.py:349: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_loc

2025-03-09 02:06:26,476 tensorboard logging path is logs/PDSC_V2/GELECTRA/G29B/3K/k1
2025-03-09 02:06:32,000 ----------------------------------------------------------------------------------------------------
2025-03-09 02:06:32,002 Model: "TextClassifier(
  (decoder): Linear(in_features=1024, out_features=3, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): CrossEntropyLoss()
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(31102, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0-23): 24 x ElectraLa

...

100%|█████████████████████████████████████████| 125/125 [00:05<00:00, 24.48it/s]

2025-03-09 03:24:01,377 Evaluating as a multi-label problem: False
2025-03-09 03:24:01,385 DEV : loss 0.23265114426612854 - f1-score (micro avg)  0.86


2025-03-09 03:24:01,932 BAD EPOCHS (no improvement): 4
2025-03-09 03:24:07,047 ----------------------------------------------------------------------------------------------------
2025-03-09 03:24:07,050 loading file logs/PDSC_V2/GELECTRA/G29B/3K/k5/best-model.pt


100%|█████████████████████████████████████████| 125/125 [00:05<00:00, 23.23it/s]

2025-03-09 03:24:25,258 Evaluating as a multi-label problem: False
2025-03-09 03:24:25,266 0.846	0.846	0.846	0.846
2025-03-09 03:24:25,267 
Results:
- F-score (micro) 0.846
- F-score (macro) 0.8456
- Accuracy 0.846

By class:
              precision    recall  f1-score   support

       PSEUD     0.8145    0.8960    0.8533       250
        ORIG     0.8844    0.7960    0.8379       250

    accuracy                         0.8460       500
   macro avg     0.8495    0.8460    0.8456       500
weighted avg     0.8495    0.8460    0.8456       500

2025-03-09 03:24:25,267 ----------------------------------------------------------------------------------------------------
